# Load Dataset and Clean It

## Known Binders to Q99814 · EPAS1 (taken from BindingDB curated by UCSD)

In [ ]:
import pandas as pd

df = pd.read_csv("Known_HIF_Binders.csv")

In [ ]:
clean_df = df[["Ligand SMILES", "IC50 (nM)"]].dropna()

# remove rows that contain '<' or '>'
has_censor = clean_df["IC50 (nM)"] \
    .astype(str) \
    .str.contains(r"[<>]")

# count how many rows will be dropped
dropped_count = has_censor.sum()
print(f"Dropping {dropped_count} rows with '<' or '>' in IC50")

# keep only the rows *without* '<' or '>'
clean_df = clean_df.loc[~has_censor].reset_index(drop=True)
clean_df

In [ ]:
import re
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski

# Clean & standardize IC50, compute pIC50
def parse_ic50_to_pic50(ic50_str):
    s = str(ic50_str).strip()
    try:
        nm = float(s)
    except ValueError:
        return np.nan

    # convert nM → M
    m = nm * 1e-9
    # compute pIC50
    pic50 = -np.log10(m)
    return pic50

clean_df["pIC50"] = clean_df["IC50 (nM)"].apply(parse_ic50_to_pic50)

# Bin into activity classes
# strong binder if pIC50 ≥ 7 (IC50 ≤ 100 nM), else weak/non-binder
threshold = 7.0
clean_df["activity_class"] = np.where(clean_df["pIC50"] >= threshold, "strong", "weak")

# Compute 2D descriptors via RDKit
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {
            "MolWt": np.nan,
            "TPSA": np.nan,
            "HBD": np.nan,
            "HBA": np.nan,
            "RotBonds": np.nan,
            "LogP": np.nan,
        }
    return {
        "MolWt": Descriptors.MolWt(mol),
        "TPSA": Descriptors.TPSA(mol),
        "HBD": Lipinski.NumHDonors(mol),
        "HBA": Lipinski.NumHAcceptors(mol),
        "RotBonds": Descriptors.NumRotatableBonds(mol),
        "LogP": Crippen.MolLogP(mol),
    }

# apply and expand into separate columns
desc_df = clean_df["Ligand SMILES"].apply(compute_descriptors).apply(pd.Series)
clean_df = pd.concat([clean_df, desc_df], axis=1)
clean_df["is_known_binder"] = True

# View the table
print(clean_df.head())

## Duds (Taken from BindingDB on Compounds that Bind to ESR1 · P03372)

In [ ]:
df = pd.read_csv("ESR1_Binders.csv")

In [ ]:
esr1_df = df[["Ligand SMILES", "IC50 (nM)"]].dropna()

# remove rows that contain '<' or '>'
has_censor = esr1_df["IC50 (nM)"] \
    .astype(str) \
    .str.contains(r"[<>]")

# count how many rows will be dropped
dropped_count = has_censor.sum()
print(f"Dropping {dropped_count} rows with '<' or '>' in IC50")

# keep only the rows *without* '<' or '>'
esr1_df = esr1_df.loc[~has_censor].reset_index(drop=True)
esr1_df.reset_index(drop=True, inplace=True)
esr1_df

In [ ]:
esr1_df["pIC50"] = esr1_df["IC50 (nM)"].apply(parse_ic50_to_pic50)

esr1_df["activity_class"] = np.where(esr1_df["pIC50"] >= threshold, "strong", "weak")


# apply and expand into separate columns
temp_df = esr1_df["Ligand SMILES"].apply(compute_descriptors).apply(pd.Series)
esr1_df = pd.concat([esr1_df, temp_df], axis=1)
esr1_df["is_known_binder"] = False


print(esr1_df.head())

## Joining the known and unknown binders

In [ ]:
all_binders = pd.concat([clean_df, esr1_df], axis=0, ignore_index=True)

In [ ]:
# mix up dataset to avoid patterns
perm = np.random.permutation(len(all_binders))
all_binders = all_binders.iloc[perm].reset_index(drop=True)

all_binders.to_csv("all_binders.csv", index=False)

## Turning CSV to jsonl

In [ ]:
import pandas as pd

all_binders = pd.read_csv("all_binders.csv")

test = all_binders.head(1)


In [ ]:
import json

def make_example(row):
    prompt = (
        "From the following information about a ligand, predict whether it can bind to the HIF-2α protein.\n\n"
        f"This ligand is represented by the SMILES string {row['Ligand SMILES']}, and exhibits an IC50 of "
        f"{row['IC50 (nM)']} nM (pIC50 = {row['pIC50']:.2f}). It has a molecular weight of {row['MolWt']:.2f} Da, "
        f"a topological polar surface area of {row['TPSA']:.2f} Å², {row['HBD']} hydrogen bond donor"
        f"{'s' if row['HBD'] != 1 else ''}, {row['HBA']} hydrogen bond acceptor"
        f"{'s' if row['HBA'] != 1 else ''}, and {row['RotBonds']} rotatable bond"
        f"{'s' if row['RotBonds'] != 1 else ''}, with a logP of {row['LogP']:.2f}.\n"
    )
    completion = "Answer: Yes, it binds to HIF-2α<eos>" if row["is_known_binder"] else "Answer: No, it doesn't bind to HIF-2α<eos>"
    return {"prompt": prompt, "bind": completion}


with open("train_hif_binding.jsonl","w") as fout:
    for _, row in all_binders.iterrows():
        ex = make_example(row)
        fout.write(json.dumps(ex) + "\n")